In [1]:
# https://www.youtube.com/watch?v=sGM1qNL5y_4&feature=youtu.be


!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook # wizualizacja pracy 

In [1]:
cd "drive/My Drive/Colab Notebooks/DW-Matrix"

/content/drive/My Drive/Colab Notebooks/DW-Matrix


In [2]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df ['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [10]:
#nowa kolumna
#df['brand_cat'] = df['brand'].factorize()[0]
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0] #normalizacja zamiana na male litery

run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [60]:
#df['brand'].shape[0]
## MODEL 

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

18280

In [0]:
def parse_features(x):
  output_dict={}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] =  value

  return output_dict  


df['features_parsed'] = df['features'].map(parse_features)


In [17]:
df['features_parsed'].head().values 

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [19]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [22]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]= df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan) 

In [24]:
df.shape

(18280, 526)

In [25]:
df.shape[0]

18280

In [28]:
#filtr procentów cech 
keys_stat ={}
for key in tqdm_notebook(keys):
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [37]:
#wybiermy te ktore maja wiecej niz 50% 
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#budujemy cechy 

df['feat_brand_cat'] = df['feat_brand'].factorize()[0] 
df['feat_color_cat'] = df['feat_color'].factorize()[0] 
df['feat_gender_cat'] = df['feat_gender'].factorize()[0] 
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0] 
df['feat_material_cat'] = df['feat_material'].factorize()[0] 


#po naiku i dodajemy do fits
df['feat_sport_cat'] = df['feat_sport'].factorize()[0] 
df['feat_style_cat'] = df['feat_style'].factorize()[0] 

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0] 



In [61]:
# uwaga :) 

df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ].shape[0]

8846

In [72]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.376136122164745, 4.210573022934744)

In [78]:
#zrobione po for  UWAGA 
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_catalog',
 'feat_shoe category',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_clothing category_cat',
 'feat_name_cat',
 'feat_terrain_cat',
 'feat_contains batteries_cat',
 'feat_number of pockets_cat',
 'feat_multi pack indicator_cat',
 'feat_colour code_cat',
 'feat_measurements:_cat',
 'feat_applicable_cat',
 'feat_materials_cat',
 'feat_safety feature_cat',
 'feat_color/finish family_cat',
 'feat_product dimensions_cat',
 'feat_power reserve_cat',
 'feat_location - city/state_cat',
 'feat_contained battery type_cat',
 'feat_case diameter_cat',
 'feat_frame shape_cat',
 'feat_sizebridge_cat',
 'feat_main color_cat',
 'feat_pronati

In [0]:
#  pozniej usunelismy 
#  'feat_manufacturer part number_cat', 'feat_color_cat',

feats = ['brand_cat','feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
feats += ['feat_rise_cat',
 'feat_international shipping?_cat',
 'feat_resolution megapixels_cat',
 'feat_band length (in.)_cat',
 'feat_����������������_cat',
 'feat_place of origin_cat',
 'feat_frame material:_cat',
 'feat_sku#_cat',
 'feat_handbag style_cat',
 'feat_fit:_cat',
 'feat_heart rate monitor_cat',
 'feat_toe type_cat',]
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [81]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)


(-57.17237810692596, 4.267730897919862)


Weight,Feature
0.2654 ± 0.0042,brand_cat
0.0991 ± 0.0088,feat_material_cat
0.0486 ± 0.0032,feat_gender_cat
0.0072 ± 0.0013,feat_style_cat
0.0031 ± 0.0004,feat_frame material:_cat
0.0005 ± 0.0001,feat_band length (in.)_cat
0.0002 ± 0.0000,feat_sport_cat
0.0000 ± 0.0000,feat_place of origin_cat
0.0000 ± 0.0000,feat_rise_cat
0 ± 0.0000,feat_����������������_cat


In [64]:
#Sprwdzamy dla czego brend
df['brand'].value_counts()


nike                       1777
puma                        609
ralph lauren                526
vans                        386
new balance                 371
                           ... 
kangaroo                      1
chopard                       1
disguise costumes             1
schott                        1
polo sport ralph lauren       1
Name: brand, Length: 1732, dtype: int64

In [65]:
df[df['brand']== 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [3]:
pwd

'/content/drive/My Drive/Colab Notebooks/DW-Matrix'

In [0]:
!git add d5/day3.ipynb